In [ ]:
############################################################################################
################# Gemal Hisuin #############################################################
############# Fundamentals of Machine Learning - TP3 #######################################
############################################################################################

FORWARD ALGORITHM

In [32]:
###### computation of the probability of an observation sequence with the forward algorithm ######


import pandas as pd
import numpy as np
 
'''
    The computed probability on the current time step would be used to derive the probability of the next time 
    step in this forward sequence algorithm. We will calculate the probability that the Hidden Markov Model 
    will be in a certain hidden state (s) at a specific time stage (t) given a series of visible states (VT).
'''
'''
Input Arguments:
        transition_probability: State transition probability matrix of dimension 3 x 3
        start_probability: Initial state distribution  of dimension 3
        emission_probability: Emission probability of dimension 3*3
        visible_State: Observation sequence of length 3

Returns:
        forward_sequence_prob: Output probability matrix of dimension 3 x 3


'''
hidden_states={'Box-1', 'Box-2', 'Box-3'}
## Assign observation states to numeric numbers {a=0; b=1; e=2} for simplicicty
visible_state = np.array((0,1,2)) # 0 represents {a}; 1 represents {b}; 2 represents {e} 

##### set the values for transition probability, emission probabilities and initial distribution.
# initial distribution    
start_probability = np.array((1/3, 
                              1/3, 
                              1/3))
 
# Transition Probabilities    
transition_probability = np.array(((1/3, 1/3, 1/3), 
                                   (1/3, 1/3, 1/3), 
                                   (1/3, 1/3, 1/3)))
# Emission Probabilities 
emission_probability = np.array(((0.5, 0.5, 0), 
                                 (0, 0.5, 0),
                                 (0, 0, 0.5)))


'''t will start from 0 and ends at T-1.
we create the forward_sequence_prob matrix with 3 Columns and T Rows.
We multiply start_probability with emission probabaility to calculate α0(0),α1(0)
Starting from index 0 we will loop through the time steps.
We will use the same method to calculate t until T-1.
Print all of the forward_sequence_prob values.
'''
def forward(visible_state, transition_probability, emission_probability, start_probability):
    forward_sequence_prob = np.zeros((visible_state.shape[0], transition_probability.shape[0]))
    forward_sequence_prob[0, :] = start_probability * emission_probability[:, visible_state[0]]
 
    for t in range(1, visible_state.shape[0]):
        for j in range(transition_probability.shape[0]):
            forward_sequence_prob[t, j] = forward_sequence_prob[t - 1].dot(transition_probability[:, j]) * emission_probability[j, visible_state[t]]
    return forward_sequence_prob
 
forward_sequence_prob = forward(visible_state, transition_probability, emission_probability, start_probability)
forward_sequence_prob = np.transpose(forward_sequence_prob)
print("   t1 ------- t2 -------- t3")

'''
The result shows the probability of a given {a,b,e} sequence.
    t1 ------- t2 -------- t3
[[0.16666667 0.02777778 0.        ]
 [0.         0.02777778 0.        ]
 [0.         0.         0.00925926]]
The forward sequance algorithm yields the same results for the given HMM observation as the results I obtained 
in the Modeling section(1) of this report. The value obtained for {a,b,e} at stage t3 on 'Modeling section' is 0.009259,
which is the same with the value obtained on this python program
 
'''
print(forward_sequence_prob)



   t1 ------- t2 -------- t3
[[0.16666667 0.02777778 0.        ]
 [0.         0.02777778 0.        ]
 [0.         0.         0.00925926]]


BACKWARD ALGORITHM

In [33]:
###### computation of the probability of an observation sequence with the backward algorithm ######


import pandas as pd
import numpy as np
 
'''
Input Arguments:
        transition_probability: State transition probability matrix of dimension 3 x 3
        emission_probability: Emission probability of dimension 3*3
        visible_State: Observation sequence of length 3

Returns:
        backward_sequence_prob: Output probability matrix of dimension 3 x 3


'''
'''
 We usually don't need to know the Backward Algorithm to solve the Likelihood problem. Its description and solution is 
 a strong smoothness measure to demonstrate that the forward algorithm functions appropriately. 
 The backward algorithm is equivalent to the forward algorithm, but it goes backward in time as the name implies. 
'''
hidden_states={'Box-1', 'Box-2', 'Box-3'}
## Assign observation states to numeric numbers {a=0; b=1; e=2} for simplicicty
visible_state = np.array((0,1,2)) # 0 represents {a}; 1 represents {b}; 2 represents {e} 
 
# Transition Probabilities    
transition_probability = np.array(((1/3, 1/3, 1/3), 
                                   (1/3, 1/3, 1/3), 
                                   (1/3, 1/3, 1/3)))
# Emission Probabilities 
emission_probability = np.array(((0.5, 0.5, 0), 
                                 (0, 0.5, 0),
                                 (0, 0, 0.5)))

 
'''
The backward variables of any state at the end of the observation series are equal to 1.
We would be able to measure the backward variable of the previous state by going backwards. 

''' 
def backward(visible_state, transition_probability, emission_probability):
    backward_sequence_prob = np.zeros((visible_state.shape[0], transition_probability.shape[0]))
 
    # set backward_sequence_prob(T) = 1
    backward_sequence_prob[visible_state.shape[0] - 1] = np.ones((transition_probability.shape[0]))
 
    # From T-1, make a backward loop.
    for t in range(visible_state.shape[0] - 2, -1, -1): # The main loop would be T-2 to 0 because of python indexing.
        for j in range(transition_probability.shape[0]):
            backward_sequence_prob[t, j] = (backward_sequence_prob[t + 1] * emission_probability[:, visible_state[t + 1]]).dot(transition_probability[j, :])
 
    return backward_sequence_prob
 
 
backward_sequence_prob = backward(visible_state, transition_probability, emission_probability)
backward_sequence_prob = np.transpose(backward_sequence_prob)
print("   t1 ----- t2 --- t3")

'''
This is accomplished by adding up three multiplication values:

For instance,
Transition probability from Box-1 to Box-1 (1/3) times the emission probability of {a} originating from 
Box-1 state at time t+1 (0.5) times the backward variable of the Sunny State at time t+1 (1) 
1/3*0.5*1 = 0.16667

Transition probability from Box-1 to Box-2 (1/3) times the emission probability of {a} originating from 
Box-2 state at time t+1 (0.0) times the backward variable of the Sunny State at time t+1 (1) 
1/3*0*1 = 0

Transition probability from Box-1 to Box-1 (1/3) times the emission probability of {a} originating from 
Box-3 state at time t+1 (0.0) times the backward variable of the Sunny State at time t+1 (1)
1/3*0*1 = 0

We will sum the three multiplication results 
0.16667+0+0 = 0.166667
0.16667  will be the probability of Box-1 at time step-2 (t2)
'''
print(backward_sequence_prob)

   t1 ----- t2 --- t3
[[0.05555556 0.16666667 1.        ]
 [0.05555556 0.16666667 1.        ]
 [0.05555556 0.16666667 1.        ]]


VITERBI ALGORITHM

In [34]:
#### The extraction of the most likely sequence of states explaining a sequence of observation with the Viterbi Algorithm.

import pandas as pd
import numpy as np

'''
Input Arguments:
        transition_probability: State transition probability matrix of dimension 3 x 3
        start_probability: Initial state distribution  of dimension 3
        emission_probability: Emission probability of dimension 3*3
        visible_State: Observation sequence of length 3

Returns:
        Optimal state sequence of length 3

'''
hidden_states={'Box-1', 'Box-2', 'Box-3'}
## Assign observation states to numeric numbers {a=0; b=1; e=2} for simplicicty
visible_state = np.array((0,1,2)) # 0 represents {a}; 1 represents {b}; 2 represents {e} 


observations = ('a', 'b', 'e')

# Equal Probabilities for the initial distribution    
start_probability = np.array((1/3, 
                              1/3, 
                              1/3))
 
# Transition Probabilities    
transition_probability = np.array(((1/3, 1/3, 1/3), 
                                   (1/3, 1/3, 1/3), 
                                   (1/3, 1/3, 1/3)))
# Emission Probabilities 
emission_probability = np.array(((0.5, 0.5, 0), 
                                 (0, 0.5, 0),
                                 (0, 0, 0.5)))

'''
Using viterbi algorithm,
We will determine what the most probable next hidden state will be at each time step (t) and each hidden state.
The aim is to represent the highest probability along a single path that ends at state t.

Steps:
- To define the state that maximizes at each time step t, we must first find the sequence of hidden states for all observations.
- We'll use backpointer to backtrack the most likely hidden state after finding the last hidden state using maximum likelihood.

'''
def viterbi(visible_state, transition_probability, emission_probability, start_probability):
    VS = visible_state.shape[0]
    TP = transition_probability.shape[0]
 
    last_s = np.zeros((VS, TP))
    last_s[0, :] = start_probability * emission_probability[:, visible_state[0]]
 
    previous_s = np.zeros((VS - 1, TP))
 
    for t in range(1, VS):
        for j in range(TP):
            probability = last_s[t - 1] + transition_probability[:, j] + emission_probability[j, visible_state[t]]
            previous_s[t - 1, j] = np.argmax(probability) # Provided the previous state at time t, this is our most probable state (1)
            last_s[t, j] = np.max(probability) # The likelihood of the most probable state of state (2)
   
    arr_list = np.zeros(VS) # Path Array
    last_state = np.argmax(last_s[VS - 1, :]) # Find the hidden state that is most probable to be the last one 
 
    arr_list[0] = last_state
 
    backtrack_index = 1
    for i in range(VS - 2, -1, -1):
        arr_list[backtrack_index] = previous_s[i, int(last_state)]
        last_state = previous_s[i, int(last_state)]
        backtrack_index += 1
        
    arr_list = np.flip(arr_list, axis=0)# As we were going backwards, flip the route array.
 
    # Convert numerical values into hidden states.
    result = []
    for bp in arr_list:
        if bp == 0:
            result.append("Box-1")
        elif bp == 1:
            result.append("Box-2")
        else:
            result.append("Box-3")
    return result

'''
When t = 2, for example, the chance of transitioning from Box 1(1) to Box 1(2) is greater than that of
transitioning from Box-1(1) to Box-3(2), so we keep track of Box-1(1) - Box-1(2) and forget track of Box-1(1) - Box-3(2).
Similarly, we do the same thing with each hidden state.

Finally we will obtain the most probable path, whcih will be 'Box-1 -> Box-1 -> Box-3'
'''
print("The most likely sequence of state is:", viterbi(visible_state, transition_probability, emission_probability, start_probability))



The most likely sequence of state is: ['Box-1', 'Box-1', 'Box-3']


BONUS

In [37]:
########## Bonus  ######### 

### implementation of Baum-Welch 


import pandas as pd
import numpy as np

### Assumption: 1. Unifrom distribution of initial probabilities 
              # 2. Uniform distribution of probabilites from hidden to visible state
'''
Input Arguments:
        
        start_probability: Initial state distribution  of dimension 3
        visible_State: Observation sequence of length 3
        General information on the expected transition and emission probabilities

Returns:
        transition_probability: State transition probability matrix of dimension 3 x 3
        emission_probability: Emission probability of dimension 3*3

'''
hidden_states={'Box-1', 'Box-2', 'Box-3'}
## Assign observation states to numeric numbers {a=0; b=1; e=2} for simplicicty
visible_state = np.array((0,1,2)) # 0 represents {a}; 1 represents {b}; 2 represents {e} 

# Transition Probabilities
transition_probability = np.ones((3, 3))
transition_probability = transition_probability / np.sum(transition_probability, axis=1)
 
# Emission Probabilities
emission_probability = np.array(((1, 1, 1), (1, 1, 1), (1, 1, 1))) # unifrom distribution of probabibilities between 
                                                                   # hidden state and observation state
emission_probability = emission_probability / np.sum(emission_probability, axis=1).reshape((-1, 1))
 
# Equal Probabilities for the starting distribution
start_probability = np.array((0.333, 0.333, 0.333))

## Forward and Backward Probabilities can be used to express the numerator of the equation.
def forward(visible_state, transition_probability, emission_probability, start_probability):
    forward_seq = np.zeros((visible_state.shape[0], transition_probability.shape[0]))
    forward_seq[0, :] = start_probability * emission_probability[:, visible_state[0]]
 
    for t in range(1, visible_state.shape[0]):
        for j in range(transition_probability.shape[0]):
            forward_seq[t, j] = forward_seq[t - 1].dot(transition_probability[:, j]) * emission_probability[j, visible_state[t]]
 
    return forward_seq
 
 
def backward(visible_state, transition_probability, emission_probability):
    backward_seq = np.zeros((visible_state.shape[0], transition_probability.shape[0]))
 
    # setting backward_seq(T) = 1
    backward_seq[visible_state.shape[0] - 1] = np.ones((transition_probability.shape[0]))
 
    # Loop in backward way from T-1 
    # To get the total joint probability for all transitions from hidden state i to hidden state j,
    # ...we must add all T together. This would be the equation's numerator.
    for t in range(visible_state.shape[0] - 2, -1, -1): # Due to python indexing the actual loop will be T-2 to 0
        for j in range(transition_probability.shape[0]):
            backward_seq[t, j] = (backward_seq[t + 1] * emission_probability[:, visible_state[t + 1]]).dot(transition_probability[j, :])
 
    return backward_seq
 
'''
The Baum-Welch Algorithm is a variant of the Expectation Maximization (EM) algorithm 
which is used to determine the transition and emission Matrix. 

Steps:
- Begin with initial transition and emission probability estimates. 
  We can either set equal probabilities at the start or define them at random.
- Calculate the expected frequency of each transition/emission.
- Based on those predictions, recalculate the probabilities of translation and emisssion (from latent variable).
- Repeat until convergence
''' 
def baum_welch(visible_state, transition_probability, emission_probability, start_probability, n_iter=100):
    VS = transition_probability.shape[0]
    T = len(visible_state)
 
    for n in range(n_iter):
        forward_seq = forward(visible_state, transition_probability, emission_probability, start_probability)
        backward_seq = backward(visible_state, transition_probability, emission_probability)
 
        xi = np.zeros((VS, VS, T - 1))
        for t in range(T - 1):
            denominator = np.dot(np.dot(forward_seq[t, :].T, transition_probability) * emission_probability[:, visible_state[t + 1]].T, backward_seq[t + 1, :])
            for i in range(VS):
                numerator = forward_seq [t, i] * transition_probability[i, :] * emission_probability[:, visible_state[t + 1]].T * backward_seq[t + 1, :].T
                xi[i, :, t] = numerator / denominator
 
        gamma = np.sum(xi, axis=1)
        transition_probability = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))
 
        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))
 
        EP = emission_probability.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(EP):
            emission_probability[:, l] = np.sum(gamma[:, visible_state == l], axis=1)
 
        emission_probability = np.divide(emission_probability, denominator.reshape((-1, 1)))
 
    return {"transition_probability":transition_probability, "emission_probability":emission_probability}
''' 
The transition and emission probability results obtained using this approach are identical to the values
presented in HMM modeling. 
'''
'''
Result: 
For a uniform distribution on initial probabailities and unifrom distrubtion from hidden state to visible state
{'transition_probability': array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]]), 
       'emission_probability': array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]])}

'''
 
print(baum_welch(visible_state, transition_probability, emission_probability, start_probability, n_iter=100))

{'transition_probability': array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]]), 'emission_probability': array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]])}
